# Exploring Data Available for Cryptocurrencies
- Main purpose: usage in ML model.

In [10]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
import time

In [ ]:
coin_id =  'solana'
cg = CoinGeckoAPI()

coin_data = cg.get_coin_by_id(id=coin_id, localization=False)
market_data = coin_data.get('market_data', {})
supply_data = {
    "circulating_supply": market_data.get("circulating_supply"),
    "total_supply": market_data.get("total_supply"),
    "max_supply": market_data.get("max_supply"),
    "market_cap_usd": market_data.get("market_cap", {}).get("usd"),
    "volume_24h_usd": market_data.get("total_volume", {}).get("usd"),
    "liquidity_score": coin_data.get("liquidity_score"),
}
developer_data = coin_data.get("developer_data", {})
community_data = coin_data.get("community_data", {})
trending = cg.get_search_trending()
trending_coins = [item['item']['id'] for item in trending['coins']]
is_trending = coin_id in trending_coins
categories = cg.get_coins_categories()
category_df = pd.DataFrame(categories)[['id', 'name', 'market_cap', 'market_cap_change_24h']]
historical = cg.get_coin_market_chart_by_id(id=coin_id, vs_currency='usd', days=2)
market_caps = historical.get('market_caps', [])
volumes = historical.get('total_volumes', [])
historical_df = pd.DataFrame({
    'timestamp': [pd.to_datetime(mc[0], unit='ms') for mc in market_caps],
    'market_cap': [mc[1] for mc in market_caps],
    'volume': [vol[1] for vol in volumes]
})
# some statrs from historical?
# calc market_cap / circulating_supply
# last 24h slopes, last 12 h, last 6 and 3

print("Supply & Market Cap:")
print(pd.Series(supply_data))

print("\nDeveloper Data:")
print(pd.Series(developer_data))

print("\nCommunity Data:")
print(pd.Series(community_data))

print("\nTrending Coins:")
print(trending_coins)
print(f"Is '{coin_id}' trending? {'yes' if is_trending else 'no'}")

print("\nTop Categories:")
print(category_df.head())

print("\nHistorical Market Cap & Volume (Last 7 days):")
print(historical_df.tail())


Supply & Market Cap:
circulating_supply    5.425669e+08
total_supply          6.097525e+08
max_supply                     NaN
market_cap_usd        1.309600e+11
volume_24h_usd        9.710675e+09
liquidity_score                NaN
dtype: float64

Developer Data:
forks                                                                      3516
stars                                                                     11071
subscribers                                                                 276
total_issues                                                               5177
closed_issues                                                              4611
pull_requests_merged                                                      23614
pull_request_contributors                                                   411
code_additions_deletions_4_weeks       {'additions': 10193, 'deletions': -5277}
commit_count_4_weeks                                                        171
last_4_weeks_comm

In [ ]:
def get_daily_vwap_last_7_days(coin_id: str, vs_currency='usd'):
    cg = CoinGeckoAPI()

    # Step 1: Fetch hourly price and volume data for 7 days
    data = cg.get_coin_market_chart_by_id(id=coin_id, vs_currency=vs_currency, days=7)
  
    # Step 2: Convert to DataFrame
    prices_df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    volumes_df = pd.DataFrame(data['total_volumes'], columns=['timestamp', 'volume'])

    # Step 3: Merge and process
    df = prices_df.copy()
    df['volume'] = volumes_df['volume']
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['date'] = df['timestamp'].dt.date

    # Step 4: Calculate VWAP per day
    df['price_volume'] = df['price'] * df['volume']
    daily_vwap = df.groupby('date').agg({
        'price_volume': 'sum',
        'volume': 'sum'
    })
    daily_vwap['vwap'] = daily_vwap['price_volume'] / daily_vwap['volume']
    daily_vwap = daily_vwap#.reset_index()[['date', 'vwap']]

    return daily_vwap


vwap_df = get_daily_vwap_last_7_days(coin_id)
print("\nVWAP calculation details (last 5 rows):")
print(vwap_df.tail())# maybe can use data values of last row, and also calc slopes over last 3, 7 days


VWAP calculation details (last 5 rows):
            price_volume        volume        vwap
date                                              
2025-09-10  4.738711e+13  2.144138e+11  221.007784
2025-09-11  5.103335e+13  2.264077e+11  225.404645
2025-09-12  7.286384e+13  3.060103e+11  238.109114
2025-09-13  5.953886e+13  2.464469e+11  241.588972
2025-09-14  5.078906e+13  2.070294e+11  245.322893


## Get Coin Name

In [11]:
cg = CoinGeckoAPI()

# Load full list once
coin_list = cg.get_coins_list()
coin_df = pd.DataFrame(coin_list)
coin_df['symbol'] = coin_df['symbol'].str.upper()

def get_best_coin_id(symbol: str):
    symbol = symbol.upper()
    matches = coin_df[coin_df['symbol'] == symbol]

    if matches.empty:
        return None

    # 1. Prefer exact match on name like 'Ethereum' for ETH
    preferred_names = {
        'ETH': 'Ethereum',
        'BTC': 'Bitcoin',
        'BNB': 'BNB',
        'SOL': 'Solana',
        'ADA': 'Cardano',
        # Add more if needed
    }

    if symbol in preferred_names:
        exact_match = matches[matches['name'] == preferred_names[symbol]]
        if not exact_match.empty:
            return exact_match.iloc[0]['id']

    # 2. Fallback: return the most popular-looking match (e.g., shortest ID)
    matches_sorted = matches.sort_values(by='id', key=lambda x: x.str.len())
    return matches_sorted.iloc[0]['id']

# Example
symbol = 'BNB'
coin_id = get_best_coin_id(symbol)
print(f"Best CoinGecko ID for symbol {symbol}: {coin_id}")

Best CoinGecko ID for symbol BNB: binancecoin
